In [1]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr 
import codac as cdc
from codac.datastore import client
import scipy.stats as stat
import sys
import os
import glob
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.external.magnumdbutils as ut
import flopter.external.readfastadc as adc
import flopter.databases.magnum as mdb
import flopter.core.magoptoffline as mgoff
import flopter.core.constants as c

In [2]:
def to_datetime(timestamp):
    return pd.to_datetime(client.timetostring(int(timestamp) & 0xffffffff00000000))

def get_time(f, print_fl=False):
    header = adc.read_header(f, print_fl=print_fl)
    return np.linspace(0, (header[c.MAGADC_NUM] / header[c.MAGADC_FREQ]), header[c.MAGADC_NUM])

def get_acquisition_length(f, print_fl=False):
    header = adc.read_header(f, print_fl=print_fl)
    return header[c.MAGADC_NUM] / header[c.MAGADC_FREQ]

In [3]:
def get_previous_data_si(db, search_name, time, numpify_fl=True):
    db_var = db.findNode(search_name)[0] # only take the first in list
    if numpify_fl:
        return np.array(db_var.getPrevious(time, unit=client.SI_UNIT))
    else:
        return db_var.getPrevious(time, unit=client.SI_UNIT)
    # Return 2 lists: timestamp list, data list

In [4]:
db = client.connect('127.0.0.1')
raw_data = db.root.chDir('_Raw')

time_range = ut.make_time_range_from_dates('2019-05-28 09:00:00', '2019-06-07 18:00:00')
plasma_states = ut.get_data_si(db, ut.PLASMA_STATE, time_range)

pulse_b_inds = []
time_ranges = []
mdb_time_ranges = []

for i in range(len(plasma_states[ut.DATA])):
    state = plasma_states[ut.DATA][i]
    if state != 18:
        continue
    pulse_b_inds.append(i)
    starttime = plasma_states[ut.TIMES][i]
    endtime = plasma_states[ut.TIMES][i+1]
    start_datetime = pd.to_datetime(client.timetostring(starttime))
    end_datetime = pd.to_datetime(client.timetostring(endtime))
    time_ranges.append([start_datetime, end_datetime])
    mdb_time_ranges.append(client.TimeRange(starttime, endtime))
#     print("{}) {}: {} - {} ({} seconds)"
#           .format(len(pulse_b_inds), 
#                   ut.human_date_str(starttime), 
#                   ut.human_time_str(starttime),
#                   ut.human_time_str(endtime), 
#                   client.timetoposix(endtime - starttime)))


In [41]:
beam_dump_down = ut.get_data_si(db, 'BeamDumpDown', time_range)
bdd_time_ranges = []
magnum_labels = [
    ut.TARGET_POS, 
    ut.TARGET_ROT, 
    ut.TARGET_TILT,
    ut.SOURCE_PUMP_SPEED,
    ut.HEATING_PUMP_SPEED,
    ut.TARGET_PUMP_SPEED,
#     ut.PUFFED_D2,
    ut.PUFFED_H2,
    ut.PUFFED_He,
    ut.SOURCE_CURRENT,
    ut.SOURCE_D2,
    ut.SOURCE_H2,
    ut.SOURCE_He,
    ut.B_FIELD,
    ut.SOURCE_POS
]
magnum_variables = {label: [] for label in magnum_labels}
    

for i in range(len(beam_dump_down[ut.DATA])):
    state = beam_dump_down[ut.DATA][i]
#     print(state)
    if state != 1:
        continue
    
    starttime = beam_dump_down[ut.TIMES][i]
    endtime = beam_dump_down[ut.TIMES][i+1]
    bdd_time_range = client.TimeRange(starttime, endtime)
    bdd_time_ranges.append(bdd_time_range)
    for label, data_list in magnum_variables.items():
#         print(label)
        data = get_previous_data_si(db, label, starttime)[ut.DATA]
        if label in [ut.TARGET_ROT, ut.TARGET_TILT]:
            data = - np.degrees(data)
        data_list.append(data)
#     tilt = - np.degrees(get_previous_data_si(db, 'TargetTiltingCurPos', starttime)[ut.DATA])
#     tilts.append(tilt)
    print("{}) {}: {} - {} ({} seconds) - Tilt = {}"
          .format(len(bdd_time_ranges), 
                  ut.human_date_str(starttime), 
                  ut.human_time_str(starttime),
                  ut.human_time_str(endtime), 
                  client.timetoposix(endtime - starttime),
                  magnum_variables[ut.TARGET_TILT][-1]))

1) 2019-05-28: 13:14:02 - 13:14:56 (54.0 seconds) - Tilt = -9.999982633619997
2) 2019-05-28: 13:16:06 - 13:27:46 (700.1999999955297 seconds) - Tilt = -9.999982633619997
3) 2019-05-28: 13:27:58 - 14:22:51 (3293.209999995306 seconds) - Tilt = -10.02381572495511
4) 2019-05-28: 14:49:40 - 14:49:45 (5.0 seconds) - Tilt = 0.02102090601917929
5) 2019-05-28: 14:56:27 - 14:56:34 (7.0 seconds) - Tilt = 0.02102090601917929
6) 2019-05-28: 14:58:23 - 14:58:30 (7.0 seconds) - Tilt = 0.02102090601917929
7) 2019-05-28: 15:19:47 - 15:20:03 (16.0 seconds) - Tilt = 0.021815330724329826
8) 2019-05-28: 15:21:52 - 15:22:07 (15.0 seconds) - Tilt = 0.021815330724329826
9) 2019-05-28: 15:25:51 - 15:25:57 (6.0 seconds) - Tilt = 0.9998884680710917
10) 2019-05-28: 15:27:32 - 15:27:39 (7.0 seconds) - Tilt = 1.9902857479602158
11) 2019-05-28: 15:31:03 - 15:31:10 (7.0 seconds) - Tilt = 1.9902857479602158
12) 2019-05-28: 15:32:35 - 15:32:41 (6.0 seconds) - Tilt = 2.973825731366059
13) 2019-05-28: 16:14:06 - 16:14:13 

In [53]:
target_positions = get_previous_data_si(db, ut.TARGET_POS, bdd_time_ranges[200].startTime)
print(target_positions[ut.DATA])
    
for i in range(len(target_positions[ut.DATA][0:1])):
    state = target_positions[ut.DATA][i]
    print(state)

-0.20000076293945312


IndexError: invalid index to scalar variable.

In [7]:
for item in raw_data.ls():
    print(item)

# print(raw_data.ls())

Adc0TrigStartFt
AdcSrcCat1Current
AdcSrcCat1Volt
AdcSrcCatHouseVolt
AdcSrcPlate1Volt
AdcSrcPlate2Volt
AdcSrcPlate3Volt
AdcSrcPlate4Volt
AdcSrcPlate5Volt
AdcSrcPlate6Volt
AdcTargetCurrent
AdcTargetSkimmerVolt
AdcTargetVolt
AnodeFlowAr
AnodeFlowArO2
AnodeFlowD2
AnodeFlowH2
AnodeFlowHe
AnodeFlowMe
AnodeFlowN2
AnodeFlowNe
AnodeFlowXe
AvantesCh1Calibration
AvantesCh1Counts
AvantesCh1DarkCounts
AvantesCh1DelTime
AvantesCh1Description
AvantesCh1IntTime
AvantesCh1NrOfAvg
AvantesCh1SmoothPx
AvantesCh2Calibration
AvantesCh2Counts
AvantesCh2DarkCounts
AvantesCh2DelTime
AvantesCh2Description
AvantesCh2IntTime
AvantesCh2NrOfAvg
AvantesCh2SmoothPx
AvantesCh3Counts
AvantesCh3DarkCounts
AvantesCh3DelTime
AvantesCh3Description
AvantesCh3IntTime
AvantesCh3NrOfAvg
AvantesCh3SmoothPx
AvantesCh4Counts
AvantesCh4DarkCounts
AvantesCh4DelTime
AvantesCh4Description
AvantesCh4IntTime
AvantesCh4NrOfAvg
AvantesCh4SmoothPx
AvantesCh5Counts
AvantesCh5DarkCounts
AvantesCh5DelTime
AvantesCh5Description
AvantesCh5IntT

In [6]:
ts_dens_prof = 'TsProfNe'
ts_dens_prof_d = 'TsProfNe_d'
ts_temp_prof = 'TsProfTe'
ts_temp_prof_d = 'TsProfTe_d'
ts_rad_coords = 'TsRadCoords'
ts_variable_names = [ts_dens_prof, ts_dens_prof_d, ts_temp_prof, ts_temp_prof_d, ts_rad_coords]
time_range = ut.make_time_range_from_dates('2019-05-28 09:00:00', '2019-06-07 18:00:00')

In [7]:
magnum_db = mdb.MagnumDB(time_range=time_range)

# all_magnum_data = magnum_db.get_data_dict(variables=raw_data.ls())
all_magnum_data = magnum_db.get_data_dict(variables=ts_variable_names)
print(len(all_magnum_data))

5


In [9]:
for variable, data in all_magnum_data.items():
    print(variable, np.shape(data))
#     print([data[1], data[1]])

TsProfNe (2, 627)
TsProfNe_d (2, 627)
TsProfTe (2, 627)
TsProfTe_d (2, 627)
TsRadCoords (2, 627)


In [11]:
np.shape(all_magnum_data[ts_dens_prof][1][0])

(25,)

In [8]:
print(client.timetostring(all_magnum_data[ts_dens_prof][0][1]))
print('_'.join(client.timetostring(int(all_magnum_data[ts_dens_prof][0][0]) & 0xffffffff00000000).split()))

dt = pd.to_datetime(client.timetostring(all_magnum_data[ts_dens_prof][0][0]))
print(dt)

print(all_magnum_data[ts_dens_prof][0][0], mdb_time_ranges[0].startTime, mdb_time_ranges[0].endTime)

2019-05-28 14:56:28.038958
2019-05-28_14:49:40
2019-05-28 14:49:40.686322
6696074692831399535 6696074693749138960 6696074710929008144


In [9]:
shot_numbers = np.zeros_like(all_magnum_data[ts_dens_prof][0]) - 1
meas = 0
current_shot = 0
# # for meas in range(len(shot_numbers)):
# for i in range(len(mdb_time_ranges)):
#     while mdb_time_ranges[i][0] <= pd.to_datetime(client.timetostring(all_magnum_data[ts_dens_prof][0][meas])) <= time_ranges[i][1]:
#         shot_numbers[meas] = i
#         meas += 1
# #         print('Yes')

for meas in range(len(shot_numbers)):
    if bdd_time_ranges[current_shot].startTime <= all_magnum_data[ts_dens_prof][0][meas] <= bdd_time_ranges[current_shot].endTime:
        shot_numbers[meas] = current_shot
    else:
        current_shot += 1
        if bdd_time_ranges[current_shot].startTime <= all_magnum_data[ts_dens_prof][0][meas] <= bdd_time_ranges[current_shot].endTime:
            shot_numbers[meas] = current_shot
#         meas -= 1
#         print('meas decremented')
print(shot_numbers)



[ -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1
  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1
  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1
  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1
  -1  -1  75  75  76  76  76  77  77  77  78  78  78  79  79  79  80  80
  80  81  81  81  82  82  82  83  83  83  84  84  84  85  85  85  86  86
  86  -1  88  88  89  89  89  90  90  90  91  91  91  92  92  92  93  93
  93  94  94  94  95  95  95  96  96  96  97  97  97  98  98  98  99  99
  99 100 100 100 101 101 101 102 102 102 103 103 103 104 104 104 105 105
 105 106 106 106 107 107 107 108 108 108 109 109 109 110 110 110 111 111
 111 112 112 112 113 113 114 114  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1
  -1  -1 127 127 128 128 128 129 129 129 130 131 132 132 132 133 134 135
 136 137 138 139 140 141 142  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1
  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -

In [14]:
for meas in range(len(shot_numbers)):
    print(meas, client.timetostring(mdb_time_ranges[meas].startTime), 
          client.timetostring(all_magnum_data[ts_dens_prof][0][meas]), 
          client.timetostring(mdb_time_ranges[meas].endTime))

0 2019-05-28 14:49:40.900000 2019-05-28 14:49:40.686322 2019-05-28 14:49:44.900000
1 2019-05-28 14:56:27.700000 2019-05-28 14:56:28.038958 2019-05-28 14:56:33.700000
2 2019-05-28 14:58:23.700000 2019-05-28 14:58:25.179711 2019-05-28 14:58:29.700000
3 2019-05-28 15:19:47.290000 2019-05-28 15:21:55.468869 2019-05-28 15:20:02.290000
4 2019-05-28 15:21:52.290000 2019-05-28 15:25:52.220423 2019-05-28 15:22:07.290000
5 2019-05-28 15:25:51.090000 2019-05-28 15:27:33.531082 2019-05-28 15:25:57.090000
6 2019-05-28 15:27:32.090000 2019-05-28 15:31:04.692460 2019-05-28 15:27:38.090000
7 2019-05-28 15:31:03.090000 2019-05-28 15:32:36.253056 2019-05-28 15:31:09.090000
8 2019-05-28 15:32:35.100000 2019-05-28 16:14:08.309271 2019-05-28 15:32:41.100000
9 2019-05-28 16:14:06.890000 2019-05-28 16:16:01.300004 2019-05-28 16:14:12.890000
10 2019-05-28 16:15:59.890000 2019-05-28 16:17:54.410738 2019-05-28 16:16:05.890000
11 2019-05-28 16:17:52.890000 2019-05-28 16:20:15.721652 2019-05-28 16:17:58.890000
12

IndexError: list index out of range

In [15]:
# print(shot_numbers)

In [10]:
ds_list = []

for i, profile in enumerate(all_magnum_data[ts_dens_prof][1]):
#     print(i, np.shape(profile), np.shape(all_magnum_data[ts_rad_coords][1][i]))
    ds_profile = xr.Dataset({'density': (['rad_coords'], all_magnum_data[ts_dens_prof][1][i]),
                             'd_density': (['rad_coords'], all_magnum_data[ts_dens_prof_d][1][i]),
                             'temperature': (['rad_coords'], all_magnum_data[ts_temp_prof][1][i] / 1.6e-19),
                             'd_temperature': (['rad_coords'], all_magnum_data[ts_temp_prof_d][1][i] / 1.6e-19),
                             },
                            coords={'rad_coords': all_magnum_data[ts_rad_coords][1][i],                                    
                                    'measurement': i,
                                    'measurement_timestamp': all_magnum_data[ts_dens_prof][0][i],
                                    'measurement': pd.to_datetime(client.timetostring(all_magnum_data[ts_dens_prof][0][i])),
                                    'shot_number': shot_numbers[i]
                                    })
    ds_list.append(ds_profile)
ds_all = xr.concat(ds_list, 'measurement')
print(ds_all)

<xarray.Dataset>
Dimensions:                (measurement: 627, rad_coords: 46)
Coordinates:
  * rad_coords             (rad_coords) float64 -36.47 -34.8 ... 37.14 38.82
  * measurement            (measurement) datetime64[ns] 2019-05-28T14:49:40.686322 ... 2019-06-07T15:10:32.317018
    measurement_timestamp  (measurement) int64 6696074692831399535 ... 6699790920288049762
    shot_number            (measurement) int64 -1 -1 -1 -1 -1 ... -1 -1 -1 -1 -1
Data variables:
    density                (measurement, rad_coords) float64 nan nan ... nan
    d_density              (measurement, rad_coords) float64 nan nan ... nan
    temperature            (measurement, rad_coords) float64 nan nan ... nan
    d_temperature          (measurement, rad_coords) float64 nan nan ... nan


In [11]:
print(ut.get_seconds(2))
print(all_magnum_data[ts_dens_prof][0][1])
print(client.timetostring(all_magnum_data[ts_dens_prof][0][1]))
print(client.timetostring(all_magnum_data[ts_dens_prof][0][1] - ut.get_seconds(2)))

8589934592
6696076442397649688
2019-05-28 14:56:28.038958
2019-05-28 14:56:26.038958


In [42]:
shot_numbers_from_ds = []
bdd_start_times = []
bdd_end_times = []
bdd_ts_fl = []
for i, shot_time_range in enumerate(bdd_time_ranges):
    start_time = to_datetime(shot_time_range.startTime - ut.get_seconds(5))
    end_time = to_datetime(shot_time_range.endTime + ut.get_seconds(5))
#     print(type(start_time), end_time)
    ds_shot = ds_all.sel(ts_time = slice(start_time, end_time))
#     ds_shot = ds_all.sel(measurement = slice(start_time, end_time))
    shot_numbers_from_ds.append(len(ds_shot.ts_time))
#     shot_numbers_from_ds.append(len(ds_shot.measurement))
    bdd_start_times.append(start_time)
    bdd_end_times.append(end_time)
    bdd_ts_fl.append(not len(ds_shot.ts_time) == 0)
#     bdd_ts_fl.append(not len(ds_shot.measurement) == 0)
#     print(i, ds_all.sel(shot_number = i)['temperature'].max())
    
shot_numbers_from_ds = np.array(shot_numbers_from_ds)
print(shot_numbers_from_ds)

bdd_ds = xr.Dataset({'shot_end_time': (['shot_number'], bdd_end_times),
                     'ts_fl': (['shot_number'], bdd_ts_fl),
                     'shot_time': (['shot_number'], bdd_start_times),
                     'shot_source_current': (['shot_number'], magnum_variables[ut.SOURCE_CURRENT]),
                     'shot_b_field': (['shot_number'], magnum_variables[ut.B_FIELD]),
                     'shot_tilt': (['shot_number'], magnum_variables[ut.TARGET_TILT]),
                     'shot_target_pos': (['shot_number'], magnum_variables[ut.TARGET_POS]),
                     'shot_target_rot': (['shot_number'], magnum_variables[ut.TARGET_ROT]),
                     'shot_target_pumpspeed': (['shot_number'], magnum_variables[ut.TARGET_PUMP_SPEED]),
                     'shot_source_pumpspeed': (['shot_number'], magnum_variables[ut.SOURCE_PUMP_SPEED]),
                     'shot_heating_pumpspeed': (['shot_number'], magnum_variables[ut.HEATING_PUMP_SPEED]),
                     'shot_source_pos': (['shot_number'], magnum_variables[ut.SOURCE_POS]),
                     'shot_deuterium_gf': (['shot_number'], magnum_variables[ut.SOURCE_D2]),
                     'shot_hydrogen_gf': (['shot_number'], magnum_variables[ut.SOURCE_H2]),
                     'shot_helium_gf': (['shot_number'], magnum_variables[ut.SOURCE_He]),
                     'shot_hydrogen_puffed': (['shot_number'], magnum_variables[ut.PUFFED_H2]),
                     'shot_helium_puffed': (['shot_number'], magnum_variables[ut.PUFFED_He]),
                     },
                    coords={'shot_number': np.arange(len(bdd_time_ranges))})
print(bdd_ds)
print(bdd_ds.where(bdd_ds.ts_fl == False, drop=True))

[0 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 3 3 3 1 1 1 1 1 1 1
 1 3 3 3 3 3 3 3 3 3 3 3 3 0 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 2 2 0 1 1 1 1 1 1 1 1 1 1 1 3 3 3 1 1 3 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 3 1 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4 4 4 4 3 3 2 2 1 0 0 0 1 1 1 1 3 3 3 3 3 3
 3 3 3 3 3 3 3 0 0 0 4 4 4 4 4 4 4 4 4 4 4 3 3 0 3 3 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 

In [15]:
np.radians(-bdd_ds['shot_target_pos'])

<xarray.DataArray 'shot_target_pos' (shot_number: 523)>
array([0.001954, 0.001954, 0.002443, ..., 0.002216, 0.002216, 0.002216])
Coordinates:
  * shot_number  (shot_number) int64 0 1 2 3 4 5 6 ... 517 518 519 520 521 522

In [16]:
zeroes = np.where(shot_numbers_from_ds == 0)
date = ut.human_date_str(bdd_time_ranges[zeroes[0][0]].startTime)
for zero in zeroes[0]:
    comparison_date = ut.human_date_str(bdd_time_ranges[zero].startTime)
    if comparison_date != date:
        date = comparison_date
        print('')
    print(zero, to_datetime(bdd_time_ranges[zero].startTime + ut.get_hours(1)), to_datetime(bdd_time_ranges[zero].endTime + ut.get_hours(1)))
print(shot_numbers.shape)

0 2019-05-28 14:14:02 2019-05-28 14:14:56
1 2019-05-28 14:16:06 2019-05-28 14:27:46
2 2019-05-28 14:27:58 2019-05-28 15:22:51
6 2019-05-28 16:19:47 2019-05-28 16:20:03
30 2019-05-28 17:57:06 2019-05-28 18:12:46

34 2019-05-29 11:50:41 2019-05-29 11:55:41
35 2019-05-29 12:10:06 2019-05-29 12:12:22
59 2019-05-29 13:59:58 2019-05-29 14:00:05
87 2019-05-29 16:08:52 2019-05-29 16:09:03

115 2019-06-03 11:53:50 2019-06-03 12:11:38
143 2019-06-03 15:50:54 2019-06-03 15:51:00
144 2019-06-03 15:51:52 2019-06-03 15:51:57
145 2019-06-03 15:52:31 2019-06-03 15:52:37
146 2019-06-03 15:53:45 2019-06-03 15:53:51
147 2019-06-03 15:54:24 2019-06-03 15:54:30
148 2019-06-03 15:55:27 2019-06-03 15:55:33
149 2019-06-03 15:56:07 2019-06-03 15:56:13
150 2019-06-03 15:57:09 2019-06-03 15:57:15
151 2019-06-03 15:57:47 2019-06-03 15:57:53
152 2019-06-03 15:58:28 2019-06-03 15:58:33
153 2019-06-03 15:59:44 2019-06-03 15:59:50
154 2019-06-03 16:00:24 2019-06-03 16:00:30
155 2019-06-03 16:01:05 2019-06-03 16:01:11

In [25]:
print(zeroes[0].shape)

(73,)


In [17]:
shot_numbers_from_ds[zeroes] = 1
print(shot_numbers_from_ds)
shot_numbers_new = []
ts_shot = []
current_shot = 0
for j, sn in enumerate(shot_numbers_from_ds):
    for i in range(sn):
        shot_numbers_new.append(current_shot)
        ts_shot.append(j not in zeroes[0])        
    current_shot += 1

print(shot_numbers_new)
print(len(shot_numbers_new), len(np.where(ts_shot)[0]))

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 3 3 1 1 1 1 1 1 1
 1 3 3 3 3 3 3 3 3 3 3 3 3 1 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 2 2 1 1 1 1 1 1 1 1 1 1 1 1 3 3 3 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4 4 4 4 3 3 2 2 1 1 1 1 1 1 1 1 3 3 3 3 3 3
 3 3 3 3 3 3 3 1 1 1 4 4 4 4 4 4 4 4 4 4 4 3 3 1 3 3 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 

In [18]:
shot_numbers_new = np.array(shot_numbers_new)
unique_shot_numbers, unique_shot_indices = np.unique((shot_numbers_new[np.where(ts_shot)]), return_index=True)
print(unique_shot_indices)

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  59  62  65  66  67  68  69  70  71  72  73  76  79  82  85
  88  91  94  97 100 103 106 109 112 115 118 121 124 127 130 133 136 139
 142 145 148 151 154 157 160 163 166 169 172 175 178 181 184 186 188 189
 190 191 192 193 194 195 196 197 198 199 202 205 208 209 210 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 246 247 248 249 250 251 252 253
 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271
 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289
 290 291 292 293 294 295 296 297 298 299 300 301 302 303 304 305 306 307
 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324 325
 326 327 328 329 330 331 332 333 334 335 336 337 33

In [20]:
FOLDERS = ('2019-05-28_Leland/',
           '2019-05-29_Leland/',
           '2019-06-03_Leland/',
           '2019-06-04_Leland/',
           '2019-06-05_Leland/',
           '2019-06-06_Leland/',
           '2019-06-07_Leland/',)
mgoff.Magoptoffline._FOLDER_STRUCTURE = '/Data/external/magnum/'
adc_files = []
adc_file_folders = []
for _folder in FOLDERS:
    os.chdir(mgoff.Magoptoffline.get_data_path() + _folder)
    adc_files.extend(glob.glob('*.adc'))
    adc_file_folders.extend([_folder] * len(glob.glob('*.adc')))
adc_files.sort()

for i, f in enumerate(adc_file_folders):
    print(i, f)

0 2019-05-28_Leland/
1 2019-05-28_Leland/
2 2019-05-28_Leland/
3 2019-05-28_Leland/
4 2019-05-28_Leland/
5 2019-05-28_Leland/
6 2019-05-28_Leland/
7 2019-05-28_Leland/
8 2019-05-28_Leland/
9 2019-05-28_Leland/
10 2019-05-28_Leland/
11 2019-05-28_Leland/
12 2019-05-28_Leland/
13 2019-05-28_Leland/
14 2019-05-28_Leland/
15 2019-05-28_Leland/
16 2019-05-28_Leland/
17 2019-05-28_Leland/
18 2019-05-28_Leland/
19 2019-05-28_Leland/
20 2019-05-28_Leland/
21 2019-05-28_Leland/
22 2019-05-28_Leland/
23 2019-05-28_Leland/
24 2019-05-28_Leland/
25 2019-05-28_Leland/
26 2019-05-28_Leland/
27 2019-05-28_Leland/
28 2019-05-28_Leland/
29 2019-05-28_Leland/
30 2019-05-28_Leland/
31 2019-05-28_Leland/
32 2019-05-28_Leland/
33 2019-05-29_Leland/
34 2019-05-29_Leland/
35 2019-05-29_Leland/
36 2019-05-29_Leland/
37 2019-05-29_Leland/
38 2019-05-29_Leland/
39 2019-05-29_Leland/
40 2019-05-29_Leland/
41 2019-05-29_Leland/
42 2019-05-29_Leland/
43 2019-05-29_Leland/
44 2019-05-29_Leland/
45 2019-05-29_Leland

In [234]:
for i, f in enumerate(adc_files):
    timestamp = adc.get_magnumdb_timestamp(f)    
    trigger_time = to_datetime(timestamp)
    print(trigger_time)

2019-05-28 09:29:58
2019-05-28 11:14:20
2019-05-28 11:21:35
2019-05-28 11:48:28
2019-05-28 13:20:35
2019-05-28 14:49:40
2019-05-28 14:56:27
2019-05-28 14:58:23
2019-05-28 15:19:47
2019-05-28 15:21:51
2019-05-28 15:25:50
2019-05-28 15:27:31
2019-05-28 15:31:02
2019-05-28 15:32:34
2019-05-28 16:14:06
2019-05-28 16:15:59
2019-05-28 16:17:52
2019-05-28 16:20:13
2019-05-28 16:22:27
2019-05-28 16:23:58
2019-05-28 16:25:27
2019-05-28 16:27:12
2019-05-28 16:29:05
2019-05-28 16:31:48
2019-05-28 16:33:03
2019-05-28 16:37:16
2019-05-28 16:39:25
2019-05-28 16:41:48
2019-05-28 16:44:41
2019-05-28 16:47:09
2019-05-28 16:49:56
2019-05-28 16:53:31
2019-05-28 16:57:05
2019-05-29 09:53:04
2019-05-29 09:57:06
2019-05-29 09:58:57
2019-05-29 10:09:26
2019-05-29 10:15:30
2019-05-29 10:17:56
2019-05-29 10:50:40
2019-05-29 11:10:06
2019-05-29 11:14:57
2019-05-29 11:21:50
2019-05-29 11:25:46
2019-05-29 11:27:53
2019-05-29 11:30:31
2019-05-29 11:36:53
2019-05-29 11:42:41
2019-05-29 11:45:26
2019-05-29 11:50:45


In [21]:
adc_timestamps = []
adc_endtimestamps = []
adc_starttimes = []
adc_endtimes = []
adc_acq_lengths = []
adc_shot_number = []
adc_freqs = []
# print(to_datetime(adc.get_magnumdb_timestamp(adc_files[0])))

for i, adc_file in enumerate(adc_files):
    timestamp = adc.get_magnumdb_timestamp(adc_file)
    
    fullpath = mgoff.Magoptoffline.get_data_path() + adc_file_folders[i] + adc_file
    with open(fullpath, 'rb') as f:
        header = adc.read_header(f, print_fl=False)
        acq_length = header[c.MAGADC_NUM] / header[c.MAGADC_FREQ]
        adc_freq = header[c.MAGADC_FREQ]
    
    start_time = to_datetime(timestamp)
    end_time = to_datetime(timestamp + ut.get_seconds(acq_length))
#     print(i, start_time, ' - ', end_time, ' - ', acq_length)
    
#     ds_shot = ds_all.sel(measurement = slice(start_time, end_time))
    trigger_shot = bdd_ds.swap_dims({'shot_number':'shot_time'}).sel(shot_time=slice(to_datetime(timestamp - ut.get_seconds(5)), to_datetime(timestamp + ut.get_seconds(acq_length + 5))))
    adc_timestamps.append(timestamp)
    adc_endtimestamps.append(timestamp + ut.get_seconds(acq_length))
    adc_starttimes.append(start_time)
    adc_endtimes.append(end_time)
    adc_acq_lengths.append(acq_length)
    adc_freqs.append(adc_freq)
    if len(trigger_shot.shot_number) > 0:
        adc_shot_number.extend(trigger_shot.shot_number.data)
    else:
        adc_shot_number.append(-1)
#     print(i, ds_all.sel(shot_number = i)['temperature'].max())
    
adc_timestamps = np.array(adc_timestamps)
adc_endtimestamps = np.array(adc_endtimestamps)
adc_acq_lengths = np.array(adc_acq_lengths)
adc_starttimes = np.array(adc_starttimes)
adc_endtimes = np.array(adc_endtimes)
adc_freqs = np.array(adc_freqs)

print(len(adc_timestamps), len(adc_acq_lengths))
adc_ds =  xr.Dataset({'adc_index': (['time'], np.arange(len(adc_timestamps))),
                      'acquisition_length': (['time'], adc_acq_lengths),
                      'adc_end_time': (['time'], adc_endtimes),
                      'adc_timestamp': (['time'], adc_timestamps),
                      'adc_end_timestamp': (['time'], adc_endtimestamps),
                      'adc_filename': (['time'], adc_files),
                      'adc_folder': (['time'], adc_file_folders),
                      'shot_number': (['time'], adc_shot_number),
                      'adc_freqs': (['time'], adc_freqs)
                     },
                     coords={'time': adc_starttimes})

527 527


In [22]:
print(adc_ds)
adc_ds_sn = adc_ds.swap_dims({'time':'shot_number'}).sortby('shot_number').sel(shot_number=slice(3,523)).rename({'time':'adc_time'})
adc_ds_sn['adc_end_timestamp'] = adc_ds_sn['adc_end_timestamp'].astype(np.int64)
# adc_ds_sn.sel(shot_number=-1).drop_dims('shot_number')
print(adc_ds_sn)

<xarray.Dataset>
Dimensions:             (time: 527)
Coordinates:
  * time                (time) datetime64[ns] 2019-05-28T09:29:58 ... 2019-06-07T15:10:30
Data variables:
    adc_index           (time) int64 0 1 2 3 4 5 6 ... 521 522 523 524 525 526
    acquisition_length  (time) float64 10.0 10.0 10.0 10.0 ... 10.0 10.0 10.0
    adc_end_time        (time) datetime64[ns] 2019-05-28T09:30:08 ... 2019-06-07T15:10:40
    adc_timestamp       (time) int64 6695992307612752513 ... 6699790911698096530
    adc_end_timestamp   (time) float64 6.696e+18 6.696e+18 ... 6.7e+18 6.7e+18
    adc_filename        (time) <U50 '2019-05-28 10h 30m 08s TT_06695992307612752513.adc' ... '2019-06-07 16h 10m 40s TT_06699790911698096530.adc'
    adc_folder          (time) <U18 '2019-05-28_Leland/' ... '2019-06-07_Leland/'
    shot_number         (time) int64 -1 -1 -1 -1 -1 3 ... 518 519 520 521 522
    adc_freqs           (time) float64 1e+04 1e+04 1e+05 ... 1e+06 1e+06 1e+06
<xarray.Dataset>
Dimensions:        

In [23]:
adc_triggers = []

for i, shot_time_range in enumerate(bdd_time_ranges):
    start_time = to_datetime(shot_time_range.startTime - ut.get_seconds(5))
    end_time = to_datetime(shot_time_range.endTime + ut.get_seconds(5))
#     print(type(start_time), end_time)
    ds_shot = adc_ds.sel(time = slice(start_time, end_time))
    adc_triggers.append(len(ds_shot.time))
#     print(i, ds_all.sel(shot_number = i)['temperature'].max())

adc_triggers = np.array(adc_triggers)
print(adc_triggers)

[0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1
 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 

In [24]:
non_singular = np.where(adc_triggers != 1)
print(len(non_singular[0]))
date = ut.human_date_str(bdd_time_ranges[zeroes[0][0]].startTime)
for ns in non_singular[0]:
    comparison_date = ut.human_date_str(bdd_time_ranges[ns].startTime)
    if comparison_date != date:
        date = comparison_date
        print('')
    print(ns, adc_triggers[ns], 
          to_datetime(bdd_time_ranges[ns].startTime + ut.get_hours(1)), 
          to_datetime(bdd_time_ranges[ns].endTime + ut.get_hours(1)), 
          client.timetoposix(bdd_time_ranges[ns].endTime - bdd_time_ranges[ns].startTime))
print(shot_numbers.shape)

31
0 0 2019-05-28 14:14:02 2019-05-28 14:14:56 54.0
2 0 2019-05-28 14:27:58 2019-05-28 15:22:51 3293.209999995306

31 3 2019-05-29 10:57:06 2019-05-29 11:11:33 867.0099999997765
103 0 2019-05-29 16:49:12 2019-05-29 16:49:23 11.0
105 0 2019-05-29 16:51:11 2019-05-29 16:51:22 11.0

115 0 2019-06-03 11:53:50 2019-06-03 12:11:38 1068.0

257 0 2019-06-04 16:48:44 2019-06-04 16:48:50 6.0
258 0 2019-06-04 16:50:10 2019-06-04 16:50:15 5.0
259 0 2019-06-04 16:51:22 2019-06-04 16:51:28 6.0
260 0 2019-06-04 16:54:20 2019-06-04 16:54:26 6.0
261 0 2019-06-04 16:56:51 2019-06-04 16:56:57 6.0
262 0 2019-06-04 16:58:34 2019-06-04 16:58:39 5.0
263 0 2019-06-04 17:00:27 2019-06-04 17:00:33 6.0
264 0 2019-06-04 17:02:07 2019-06-04 17:02:13 6.0
265 0 2019-06-04 17:04:11 2019-06-04 17:04:17 6.0
266 0 2019-06-04 17:05:46 2019-06-04 17:05:52 6.0
267 0 2019-06-04 17:09:41 2019-06-04 17:09:47 6.0
268 0 2019-06-04 17:11:16 2019-06-04 17:11:22 6.0
269 0 2019-06-04 17:14:04 2019-06-04 17:14:10 6.0
270 0 2019-06-0

In [33]:
adc_calibrations = []
adc_bdd_counts = []
print(ds_all)
print(len(adc_files))

for i, adc_file in enumerate(adc_files):
    timestamp = adc.get_magnumdb_timestamp(adc_file)    
    trigger_time = to_datetime(timestamp)
    lobound_time = to_datetime(timestamp - ut.get_seconds(10))
    hibound_time = to_datetime(timestamp + ut.get_seconds(10))
    trigger_shot = bdd_ds.sel(time=slice(lobound_time, hibound_time))['time']
#     print(trigger_shot)
    adc_bdd_counts.append(len(trigger_shot))
    if len(trigger_shot) > 0:
        continue
    adc_calibrations.append(trigger_time)
#     ds_shot = adc_ds.sel(time = slice(start_time, end_time))
#     adc_triggers.append(len(ds_shot.time))
# #     print(i, ds_all.sel(shot_number = i)['temperature'].max())

adc_calibrations = np.array(adc_calibrations)
adc_bdd_counts = np.array(adc_bdd_counts)
print(adc_calibrations.shape)
print(adc_bdd_counts)

<xarray.Dataset>
Dimensions:           (ts_radial_pos: 46, ts_time: 627)
Coordinates:
  * ts_radial_pos     (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
    ts_number         (ts_time) int64 0 1 2 3 4 5 6 ... 621 622 623 624 625 626
    ts_timestamp      (ts_time) int64 6696074692831399535 ... 6699790920288049762
  * ts_time           (ts_time) datetime64[ns] 2019-05-28T14:49:40.686322 ... 2019-06-07T15:10:32.317018
    shot_number       (ts_time) int64 3 4 5 7 8 9 10 ... 517 518 519 520 521 522
Data variables:
    ts_density        (ts_time, ts_radial_pos) float64 nan nan ... 1.481e+19 nan
    ts_d_density      (ts_time, ts_radial_pos) float64 nan nan ... 1.543e+18 nan
    ts_temperature    (ts_time, ts_radial_pos) float64 nan nan ... 0.08128 nan
    ts_d_temperature  (ts_time, ts_radial_pos) float64 nan nan ... 0.007526 nan
527


ValueError: dimensions or multi-index levels ['time'] do not exist

In [26]:
adc_bdd_non_singular = np.where(adc_bdd_counts != 1)
print(len(adc_bdd_non_singular[0]))
date = ut.human_date_str(bdd_time_ranges[zeroes[0][0]].startTime)
for ns in adc_bdd_non_singular[0]:
#     print(ns, adc_files[ns], adc.get_magnumdb_timestamp(adc_files[ns]), ut.human_date_str(adc.get_magnumdb_timestamp(adc_files[ns])))
    comparison_date = ut.human_date_str(adc.get_magnumdb_timestamp(adc_files[ns]))
    if comparison_date != date:
        date = comparison_date
        print('')
    print(ns, adc_bdd_counts[ns], 
          to_datetime(adc.get_magnumdb_timestamp(adc_files[ns]) + ut.get_hours(1)))
# print(shot_numbers.shape)

1


IndexError: list index out of range

In [27]:
ds_list = []
shot_numbers_ts = shot_numbers_new[np.where(ts_shot)]

for i, profile in enumerate(all_magnum_data[ts_dens_prof][1]):
#     print(i, np.shape(profile), np.shape(all_magnum_data[ts_rad_coords][1][i]))
    ds_profile = xr.Dataset({'ts_density': (['ts_radial_pos'], all_magnum_data[ts_dens_prof][1][i]),
                             'ts_d_density': (['ts_radial_pos'], all_magnum_data[ts_dens_prof_d][1][i]),
                             'ts_temperature': (['ts_radial_pos'], all_magnum_data[ts_temp_prof][1][i] / 1.6e-19),
                             'ts_d_temperature': (['ts_radial_pos'], all_magnum_data[ts_temp_prof_d][1][i] / 1.6e-19),
                             },
                            coords={'ts_radial_pos': all_magnum_data[ts_rad_coords][1][i],                                    
                                    'ts_number': i,
                                    'ts_timestamp': all_magnum_data[ts_dens_prof][0][i],
                                    'ts_time': pd.to_datetime(client.timetostring(all_magnum_data[ts_dens_prof][0][i])),
                                    'shot_number': shot_numbers_ts[i]
                                    })
    ds_list.append(ds_profile)
ds_all = xr.concat(ds_list, 'ts_time')
# ds_all['shot_number'] = shot_numbers_new
print(ds_all)

<xarray.Dataset>
Dimensions:           (ts_radial_pos: 46, ts_time: 627)
Coordinates:
  * ts_radial_pos     (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
    ts_number         (ts_time) int64 0 1 2 3 4 5 6 ... 621 622 623 624 625 626
    ts_timestamp      (ts_time) int64 6696074692831399535 ... 6699790920288049762
  * ts_time           (ts_time) datetime64[ns] 2019-05-28T14:49:40.686322 ... 2019-06-07T15:10:32.317018
    shot_number       (ts_time) int64 3 4 5 7 8 9 10 ... 517 518 519 520 521 522
Data variables:
    ts_density        (ts_time, ts_radial_pos) float64 nan nan ... 1.481e+19 nan
    ts_d_density      (ts_time, ts_radial_pos) float64 nan nan ... 1.543e+18 nan
    ts_temperature    (ts_time, ts_radial_pos) float64 nan nan ... 0.08128 nan
    ts_d_temperature  (ts_time, ts_radial_pos) float64 nan nan ... 0.007526 nan


In [28]:
def stderr_ufunc(x):
    return np.std(x, ac) / np.sqrt(np.count(x))

def stderr_applyfunc(x, dim='ts_time'):
    return x.std(dim) / np.sqrt(x.count(dim))

def stderr(x, dim):
    return xrr.apply_ufunc(stderr_ufunc, x, input_core_dims=[[dim]], dask='parallelised')


In [29]:
# Attempt at concatting along shot_number
ts_groupedby_shot_number = [group.drop('shot_number') for label, group in list(ds_all.groupby('shot_number'))]

shot_number_dim = xr.DataArray(unique_shot_numbers, dims=['shot_number'], name='shot_number')
ts_shotnumber_ds = xr.concat(ts_groupedby_shot_number, dim=shot_number_dim)

ts_shotnumber_ds.sel(shot_number=261)['ts_density']

<xarray.DataArray 'ts_density' (ts_time: 627, ts_radial_pos: 46)>
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])
Coordinates:
  * ts_time        (ts_time) datetime64[ns] 2019-05-28T14:49:40.686322 ... 2019-06-07T15:10:32.317018
  * ts_radial_pos  (ts_radial_pos) float64 -36.47 -34.8 -33.13 ... 37.14 38.82
    ts_number      (ts_time) float64 nan nan nan nan nan ... nan nan nan nan nan
    ts_timestamp   (ts_time) float64 nan nan nan nan nan ... nan nan nan nan nan
    shot_number    int64 261

In [30]:
shot_number_ts_times = ds_all.isel(ts_time=unique_shot_indices)['shot_number'].swap_dims({'ts_time':'shot_number'})
print(shot_number_ts_times)

<xarray.DataArray 'shot_number' (shot_number: 450)>
array([  3,   4,   5, ..., 520, 521, 522])
Coordinates:
    ts_number     (shot_number) int64 0 1 2 3 4 5 6 ... 621 622 623 624 625 626
    ts_timestamp  (shot_number) int64 6696074692831399535 ... 6699790920288049762
    ts_time       (shot_number) datetime64[ns] 2019-05-28T14:49:40.686322 ... 2019-06-07T15:10:32.317018
  * shot_number   (shot_number) int64 3 4 5 7 8 9 10 ... 517 518 519 520 521 522


In [31]:
# Average across shot_number to get a single averaged profile per shot
ts_shots_mean = ds_all[['ts_density', 'ts_temperature']].groupby('shot_number').mean('ts_time')
ts_shots_stderr = ds_all[['ts_d_density', 'ts_d_temperature']].groupby('shot_number').apply(stderr_applyfunc)

ts_dataset = xr.merge([ts_shots_mean, ts_shots_stderr]).assign_coords(shot_number=shot_number_ts_times)
print(ts_dataset)

/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:159: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1628: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


<xarray.Dataset>
Dimensions:           (shot_number: 450, ts_radial_pos: 46)
Coordinates:
  * ts_radial_pos     (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
  * shot_number       (shot_number) int64 3 4 5 7 8 9 ... 518 519 520 521 522
    ts_number         (shot_number) int64 0 1 2 3 4 5 ... 622 623 624 625 626
    ts_timestamp      (shot_number) int64 6696074692831399535 ... 6699790920288049762
    ts_time           (shot_number) datetime64[ns] 2019-05-28T14:49:40.686322 ... 2019-06-07T15:10:32.317018
Data variables:
    ts_density        (shot_number, ts_radial_pos) float64 nan nan ... nan
    ts_temperature    (shot_number, ts_radial_pos) float64 nan nan ... nan
    ts_d_density      (shot_number, ts_radial_pos) float64 nan nan ... 0.0 nan
    ts_d_temperature  (shot_number, ts_radial_pos) float64 nan nan ... 0.0 nan


In [32]:
ts_dataset = ts_dataset.assign({'ts_temp_max':ts_dataset.max('ts_radial_pos')['ts_temperature']})
ts_dataset = ts_dataset.assign({'ts_dens_max':ts_dataset.max('ts_radial_pos')['ts_density']})
print(ts_dataset)

<xarray.Dataset>
Dimensions:           (shot_number: 450, ts_radial_pos: 46)
Coordinates:
  * ts_radial_pos     (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
  * shot_number       (shot_number) int64 3 4 5 7 8 9 ... 518 519 520 521 522
    ts_number         (shot_number) int64 0 1 2 3 4 5 ... 622 623 624 625 626
    ts_timestamp      (shot_number) int64 6696074692831399535 ... 6699790920288049762
    ts_time           (shot_number) datetime64[ns] 2019-05-28T14:49:40.686322 ... 2019-06-07T15:10:32.317018
Data variables:
    ts_density        (shot_number, ts_radial_pos) float64 nan nan ... nan
    ts_temperature    (shot_number, ts_radial_pos) float64 nan nan ... nan
    ts_d_density      (shot_number, ts_radial_pos) float64 nan nan ... 0.0 nan
    ts_d_temperature  (shot_number, ts_radial_pos) float64 nan nan ... 0.0 nan
    ts_temp_max       (shot_number) float64 0.2624 0.3448 0.4389 ... 1.365 1.151
    ts_dens_max       (shot_number) float64 2.188e+19 2.38e+19 ... 3.083e+20


In [38]:
ts_dataset.swap_dims({'shot_number':'ts_time'}).sel(ts_time=slice('2019-06-04T15', '2019-06-04T16:55'))['ts_temp_max'].plot.line(x='shot_number')

In [459]:
ts_dataset.sel(shot_number=260)

<xarray.Dataset>
Dimensions:           (ts_radial_pos: 46)
Coordinates:
  * ts_radial_pos     (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
    shot_number       int64 260
    ts_number         int64 295
    ts_timestamp      int64 6698688961567451721
    ts_time           datetime64[ns] 2019-06-04T15:54:22.558821
Data variables:
    ts_density        (ts_radial_pos) float64 nan nan nan nan ... nan nan nan
    ts_temperature    (ts_radial_pos) float64 nan nan nan nan ... nan nan nan
    ts_d_density      (ts_radial_pos) float64 nan nan nan nan ... nan nan nan
    ts_d_temperature  (ts_radial_pos) float64 nan nan nan nan ... nan nan nan
    ts_temp_max       float64 10.34
    ts_dens_max       float64 1.17e+20

In [31]:
plt.figure()
ds_all.max('rad_coords')['temperature'].plot.line()

In [527]:
plt.figure()
plt.errorbar(ds_all['ts_radial_pos'], ds_all['ts_temperature'], yerr=ds_all['ts_d_temperature'])
plt.show()

ValueError: shape mismatch: objects cannot be broadcast to a single shape

In [33]:
ds_all.to_netcdf('ts_data.nc', engine='h5netcdf')

## Lets see if we can make it in one go

In [34]:
conv_dtype = {
    'adc_index': np.int64, 
    'adc_time': np.object, 
    'adc_timestamp': np.int64,
    'adc_4_coax': np.int64,
    'adc_4_probe': np.object,
    'adc_4_shunt_resistance': np.float64,
    'adc_5_coax': np.int64,
    'adc_5_probe': np.object,
    'adc_5_shunt_resistance': np.float64,
    'adc_calibration_index': np.object,
    'adc_calibration_fl': np.bool,
    'shot_fl': np.bool
}
probe_configuration_data = pd.read_csv('/home/jleland/Data/external/magnum/magnum2_probe-configurations.csv', 
                                       dtype=conv_dtype, 
                                       parse_dates=['adc_time'],
                                       index_col='adc_index',
                                       usecols=list(conv_dtype.keys())
                                      ).to_xarray()

print(probe_configuration_data)

<xarray.Dataset>
Dimensions:                 (adc_index: 527)
Coordinates:
  * adc_index               (adc_index) int64 0 1 2 3 4 ... 522 523 524 525 526
Data variables:
    adc_time                (adc_index) datetime64[ns] 2019-05-28T09:29:58 ... 2019-06-07T15:10:30
    adc_timestamp           (adc_index) int64 6695992307612752513 ... 6699790911698096530
    adc_4_coax              (adc_index) int64 1 1 1 1 1 1 1 1 ... 3 3 3 3 3 3 3
    adc_4_probe             (adc_index) object 'S' 'S' 'S' 'S' ... 'S' 'S' 'S'
    adc_4_shunt_resistance  (adc_index) float64 10.0 10.0 10.0 ... 1.0 1.0 1.0
    adc_5_coax              (adc_index) int64 2 2 2 2 2 2 2 2 ... 4 4 4 4 4 4 4
    adc_5_probe             (adc_index) object 'L' 'L' 'L' 'L' ... 'L' 'L' 'L'
    adc_5_shunt_resistance  (adc_index) float64 10.0 10.0 10.0 ... 1.0 1.0 1.0
    adc_calibration_index   (adc_index) object '-' '0' '0' ... '502' '502' '502'
    adc_calibration_fl      (adc_index) bool True False False ... False False
    s

In [35]:
print(adc_ds_sn.swap_dims({'shot_number':'adc_index'}).sel(adc_index=261))
print(adc_ds_sn)

<xarray.Dataset>
Dimensions:             ()
Coordinates:
    adc_index           int64 261
    shot_number         int64 246
    adc_time            datetime64[ns] 2019-06-04T15:09:42
Data variables:
    acquisition_length  float64 8.0
    adc_end_time        datetime64[ns] 2019-06-04T15:09:50
    adc_timestamp       int64 6698677450762557670
    adc_end_timestamp   int64 6698677485122295808
    adc_filename        <U50 '2019-06-04 16h 09m 50s TT_06698677450762557670.adc'
    adc_folder          <U18 '2019-06-04_Leland/'
    adc_freqs           float64 1e+06
<xarray.Dataset>
Dimensions:             (shot_number: 493)
Coordinates:
  * shot_number         (shot_number) int64 3 4 5 6 7 8 ... 518 519 520 521 522
    adc_time            (shot_number) datetime64[ns] 2019-05-28T14:49:40 ... 2019-06-07T15:10:30
Data variables:
    adc_index           (shot_number) int64 5 6 7 8 9 10 ... 522 523 524 525 526
    acquisition_length  (shot_number) float64 5.0 8.0 8.0 8.0 ... 10.0 10.0 10.0
    adc

In [36]:
adc_index_array = adc_ds_sn.swap_dims({'shot_number':'adc_index'})['adc_index']

probe_dataset = probe_configuration_data.sel(adc_index=adc_index_array).set_coords('adc_time')
adc_dataset = adc_ds_sn.swap_dims({'shot_number':'adc_index'})
# probe_dataset['adc_calibration_index'] = probe_dataset['adc_calibration_index'].astype(np.int64)
print(probe_dataset)
print(adc_dataset)
# print(data_array.values)

<xarray.Dataset>
Dimensions:                 (adc_index: 493)
Coordinates:
  * adc_index               (adc_index) int64 5 6 7 8 9 ... 522 523 524 525 526
    adc_time                (adc_index) datetime64[ns] 2019-05-28T14:49:40 ... 2019-06-07T15:10:30
    shot_number             (adc_index) int64 3 4 5 6 7 ... 518 519 520 521 522
Data variables:
    adc_timestamp           (adc_index) int64 6696074692831398075 ... 6699790911698096530
    adc_4_coax              (adc_index) int64 1 1 1 1 1 1 1 1 ... 3 3 3 3 3 3 3
    adc_4_probe             (adc_index) object 'S' 'S' 'S' 'S' ... 'S' 'S' 'S'
    adc_4_shunt_resistance  (adc_index) float64 10.0 10.0 10.0 ... 1.0 1.0 1.0
    adc_5_coax              (adc_index) int64 2 2 2 2 2 2 2 2 ... 4 4 4 4 4 4 4
    adc_5_probe             (adc_index) object 'L' 'L' 'L' 'L' ... 'L' 'L' 'L'
    adc_5_shunt_resistance  (adc_index) float64 10.0 10.0 10.0 ... 1.0 1.0 1.0
    adc_calibration_index   (adc_index) object '0' '0' '0' ... '502' '502' '502'
   

In [37]:
adc_probe_dataset = xr.merge([probe_dataset, adc_dataset]).swap_dims({'adc_index': 'shot_number'})
print(adc_probe_dataset)
print(adc_probe_dataset['adc_index'].values)
print(adc_probe_dataset.swap_dims({'shot_number':'adc_index'}).sel(adc_index=238))

<xarray.Dataset>
Dimensions:                 (shot_number: 493)
Coordinates:
    adc_index               (shot_number) int64 5 6 7 8 9 ... 523 524 525 526
    adc_time                (shot_number) datetime64[ns] 2019-05-28T14:49:40 ... 2019-06-07T15:10:30
  * shot_number             (shot_number) int64 3 4 5 6 7 ... 519 520 521 522
Data variables:
    adc_timestamp           (shot_number) int64 6696074692831398075 ... 6699790911698096530
    adc_4_coax              (shot_number) int64 1 1 1 1 1 1 1 ... 3 3 3 3 3 3 3
    adc_4_probe             (shot_number) object 'S' 'S' 'S' 'S' ... 'S' 'S' 'S'
    adc_4_shunt_resistance  (shot_number) float64 10.0 10.0 10.0 ... 1.0 1.0 1.0
    adc_5_coax              (shot_number) int64 2 2 2 2 2 2 2 ... 4 4 4 4 4 4 4
    adc_5_probe             (shot_number) object 'L' 'L' 'L' 'L' ... 'L' 'L' 'L'
    adc_5_shunt_resistance  (shot_number) float64 10.0 10.0 10.0 ... 1.0 1.0 1.0
    adc_calibration_index   (shot_number) object '0' '0' '0' ... '502' '50

In [302]:
calibrations = probe_configuration_data.where(probe_configuration_data.adc_calibration_fl, drop=True)
print(calibrations)

<xarray.Dataset>
Dimensions:                 (adc_index: 30)
Coordinates:
  * adc_index               (adc_index) int64 0 33 35 36 81 ... 473 487 489 502
Data variables:
    adc_time                (adc_index) datetime64[ns] 2019-05-28T09:29:58 ... 2019-06-07T14:14:23
    adc_timestamp           (adc_index) float64 6.696e+18 6.696e+18 ... 6.7e+18
    adc_4_coax              (adc_index) float64 1.0 1.0 1.0 1.0 ... 1.0 1.0 3.0
    adc_4_probe             (adc_index) object 'S' 'S' 'S' 'S' ... 'R' 'R' 'S'
    adc_4_shunt_resistance  (adc_index) float64 10.0 10.0 10.0 ... 1.0 1.0 1.0
    adc_5_coax              (adc_index) float64 2.0 2.0 2.0 2.0 ... 2.0 2.0 4.0
    adc_5_probe             (adc_index) object 'L' 'L' 'L' 'L' ... 'B' 'B' 'L'
    adc_5_shunt_resistance  (adc_index) float64 10.0 10.0 10.0 ... 1.0 1.0 1.0
    adc_calibration_index   (adc_index) object '-' '-' '-' '-' ... '-' '-' '-'
    adc_calibration_fl      (adc_index) float64 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0
    shot_fl     

In [357]:
print("ADC Data Set (with shot numbers) \n", adc_ds_sn, "\n")
print("Probe configuration Data Set \n", probe_dataset, "\n")
print("Thomson Scattering Data Set \n", ts_dataset, "\n")
print("Shot (beam dump down) Data Set \n", bdd_ds, "\n")

ADC Data Set (with shot numbers) 
 <xarray.Dataset>
Dimensions:             (shot_number: 493)
Coordinates:
  * shot_number         (shot_number) int64 3 4 5 6 7 8 ... 518 519 520 521 522
    adc_time            (shot_number) datetime64[ns] 2019-05-28T14:49:40 ... 2019-06-07T15:10:30
Data variables:
    adc_index           (shot_number) int64 5 6 7 8 9 10 ... 522 523 524 525 526
    acquisition_length  (shot_number) float64 5.0 8.0 8.0 8.0 ... 10.0 10.0 10.0
    adc_end_time        (shot_number) datetime64[ns] 2019-05-28T14:49:45 ... 2019-06-07T15:10:40
    adc_timestamp       (shot_number) int64 6696074692831398075 ... 6699790911698096530
    adc_end_timestamp   (shot_number) int64 6696074714306234368 ... 6699790954647769088 

Probe configuration Data Set 
 <xarray.Dataset>
Dimensions:                 (adc_index: 493)
Coordinates:
  * adc_index               (adc_index) int64 5 6 7 8 9 ... 522 523 524 525 526
    adc_time                (adc_index) datetime64[ns] 2019-05-28T14:49:40 .

In [43]:
dtype_conversions = {
    'shot_number': np.int64,
    'adc_index': np.int64,
    'ts_fl': np.bool,
    'adc_timestamp': np.int64,
    'adc_4_coax': np.int64,
    'adc_4_probe': np.object,
    'adc_4_shunt_resistance': np.float64,
    'adc_5_coax': np.int64,
    'adc_5_probe': np.object,
    'adc_5_shunt_resistance': np.float64,
    'adc_calibration_index': np.object,
    'adc_calibration_fl': np.bool,
    'shot_fl': np.bool,
    'acquisition_length': np.float64,
    'adc_end_timestamp': np.int64
}
all_dataset = xr.merge([ts_dataset, bdd_ds, adc_probe_dataset]).assign_attrs({'dtype_conversions': dtype_conversions})
# for variable, desired_dtype in dtype_conversions.items():
#     all_dataset[variable] = all_dataset[variable].astype(desired_dtype)
print(all_dataset)

<xarray.Dataset>
Dimensions:                 (shot_number: 523, ts_radial_pos: 46)
Coordinates:
  * shot_number             (shot_number) int64 0 1 2 3 4 ... 519 520 521 522
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
    ts_number               (shot_number) float64 nan nan nan ... 625.0 626.0
    ts_timestamp            (shot_number) float64 nan nan ... 6.7e+18 6.7e+18
    ts_time                 (shot_number) datetime64[ns] NaT ... 2019-06-07T15:10:32.317018
    adc_index               (shot_number) float64 nan nan nan ... 525.0 526.0
    adc_time                (shot_number) datetime64[ns] NaT ... 2019-06-07T15:10:30
Data variables:
    ts_density              (shot_number, ts_radial_pos) float64 nan nan ... nan
    ts_temperature          (shot_number, ts_radial_pos) float64 nan nan ... nan
    ts_d_density            (shot_number, ts_radial_pos) float64 nan nan ... nan
    ts_d_temperature        (shot_number, ts_radial_pos) float64 nan nan ..

In [524]:
print(all_dataset.where(all_dataset.adc_index==238, drop=True)) #[['adc_time', 'ts_time', 'shot_time']])
print(all_dataset.sel(shot_number=260))
# print(adc_probe_dataset.sel(shot_number=261))
fig, axes = plt.subplots(1)
all_dataset.sel(shot_number=260)['ts_temperature'].plot.line(ax=axes)
all_dataset.sel(shot_number=260)['ts_density'].plot.line(ax=axes.twinx(), color='orange')
axes.legend(['Temperature', 'Density'])
plt.show()

<xarray.Dataset>
Dimensions:                 (shot_number: 1, ts_radial_pos: 46)
Coordinates:
  * shot_number             (shot_number) int64 225
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
    ts_number               (shot_number) float64 260.0
    ts_timestamp            (shot_number) float64 6.699e+18
    ts_time                 (shot_number) datetime64[ns] 2019-06-04T13:54:39.620367
    adc_index               (shot_number) float64 238.0
    adc_time                (shot_number) datetime64[ns] 2019-06-04T13:54:37
Data variables:
    ts_density              (shot_number, ts_radial_pos) float64 nan nan ... nan
    ts_temperature          (shot_number, ts_radial_pos) float64 nan nan ... nan
    ts_d_density            (shot_number, ts_radial_pos) float64 nan nan ... nan
    ts_d_temperature        (shot_number, ts_radial_pos) float64 nan nan ... nan
    ts_temp_max             (shot_number) float64 7.874
    ts_dens_max             (shot_number) fl

In [56]:
all_dataset.attrs = {}
all_dataset.to_netcdf('all_meta_data.nc')

In [7]:
current_dir = os.getcwd()
os.chdir('/home/jleland/Data/')
all_dataset = xr.open_dataset('all_meta_data.nc')
os.chdir(current_dir)

In [39]:
os.chdir('/home/jleland/Data')

In [55]:
os.chdir('/home/jleland/Data/external/magnum')

In [8]:
adc_shotnumber_index = adc_index_array.swap_dims({'adc_index':'shot_number'})['shot_number'].values
print(adc_shotnumber_index)
# for i in adc_shotnumber_index:
#     print(f'{i}, ')
all_dataset.sel(shot_number=adc_shotnumber_index)['shot_number']

NameError: name 'adc_index_array' is not defined

In [9]:
adc_shotnumber_index = all_dataset.where(np.isfinite(all_dataset['adc_index']), drop=True)['shot_number'].values

print(all_dataset.max('ts_radial_pos').sel(shot_number=adc_shotnumber_index).to_dataframe().iloc[0].to_dict())

# Combine 
data_dicts = [value for label, value in all_dataset.max('ts_radial_pos').sel(shot_number=adc_shotnumber_index[0]).to_dict().items() if label in ['coords', 'data_vars']]
data_dict = {key: value['data'] for key, value in {**data_dicts[0], **data_dicts[1]}.items()}
print(data_dict)

{'adc_filename': '2019-05-28 15h 49m 45s TT_06696074692831398075.adc', 'ts_density': 2.18802022279052e+19, 'ts_temperature': 0.26238938453554556, 'ts_d_density': 0.0, 'ts_d_temperature': 0.0, 'adc_folder': '2019-05-28_Leland/', 'adc_calibration_index': '0', 'adc_4_probe': 'S', 'adc_5_probe': 'L', 'ts_number': 0.0, 'ts_timestamp': 6.6960746928314e+18, 'ts_time': Timestamp('2019-05-28 14:49:40.686322'), 'ts_temp_max': 0.26238938453554556, 'ts_dens_max': 2.18802022279052e+19, 'shot_end_time': Timestamp('2019-05-28 14:49:50'), 'ts_fl': True, 'shot_time': Timestamp('2019-05-28 14:49:35'), 'adc_index': 5.0, 'adc_time': Timestamp('2019-05-28 14:49:40'), 'adc_timestamp': 6.696074692831398e+18, 'adc_4_coax': 1.0, 'adc_4_shunt_resistance': 10.0, 'adc_5_coax': 2.0, 'adc_5_shunt_resistance': 10.0, 'adc_calibration_fl': 0.0, 'shot_fl': 1.0, 'acquisition_length': 5.0, 'adc_end_time': Timestamp('2019-05-28 14:49:45'), 'adc_end_timestamp': 6.696074714306234e+18, 'adc_freqs': 100000.0}
{'shot_number': 

/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:220: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwds)
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:220: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwds)
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:220: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwds)
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:220: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwds)


In [14]:
shot_dataarray = all_dataset.max('ts_radial_pos').sel(shot_number=adc_shotnumber_index[0])
print(shot_dataarray)
int((shot_dataarray['shot_end_time'].values - shot_dataarray['adc_end_time'].values) / np.timedelta64(1, 's'))

<xarray.Dataset>
Dimensions:                 ()
Coordinates:
    shot_number             int32 3
    ts_number               float64 ...
    ts_timestamp            float64 ...
    ts_time                 datetime64[ns] ...
    adc_index               float64 5.0
    adc_time                datetime64[ns] ...
Data variables:
    adc_filename            object '2019-05-28 15h 49m 45s TT_06696074692831398075.adc'
    ts_density              float64 2.188e+19
    ts_temperature          float64 0.2624
    ts_d_density            float64 0.0
    ts_d_temperature        float64 0.0
    adc_folder              object '2019-05-28_Leland/'
    adc_calibration_index   object '0'
    adc_4_probe             object 'S'
    adc_5_probe             object 'L'
    ts_temp_max             float64 0.2624
    ts_dens_max             float64 2.188e+19
    shot_end_time           datetime64[ns] 2019-05-28T14:49:50
    ts_fl                   bool True
    shot_time               datetime64[ns] 2019-05-28

/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:220: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwds)


5

In [18]:
variables_oi = ['ts_temp_max', 'ts_dens_max', 'adc_4_probe', 'adc_4_coax', 'adc_4_shunt_resistance', 'adc_5_probe', 'adc_5_coax', 'adc_5_shunt_resistance', 'adc_filename', 'adc_folder', 'adc_timestamp']

print(shot_dataarray[variables_oi[0]].values)

0.26238938453554556


In [29]:
CABLE_RESISTANCES = [1.7, 1.7, 1.7, 1.6]
PROBE_RESISTANCES = {
    'S': 1.0,
    'L': 1.0,
    'B': 1.0,
    'R': 1.8
}
DESIRED_DATARATE = 10000

def get_sweep_range(shot_end_time, adc_end_time, acq_length, adc_freq):
    end_index = int(acq_length * adc_freq * 0.01)
    if shot_end_time < adc_end_time:
        end_index = int((acq_length - ((shot_end_time - adc_end_time) / np.timedelta64(1, 's')) - 2) * adc_freq * 0.01)
    return 1, end_index

# shot_dataarray = shot_dataset.sel(shot_number=shot_number)
shot_number = 0
folder = shot_dataarray['adc_folder'].values
print(folder)

adc_file = shot_dataarray['adc_filename'].values
output_tag = str(int(shot_dataarray['adc_timestamp'].values))
print(adc_file, output_tag)

ts_temp = shot_dataarray['ts_temp_max'].values
ts_dens = shot_dataarray['ts_dens_max'].values
print(ts_temp, ts_dens)

probe_designations = (str(shot_dataarray['adc_4_probe'].values), str(shot_dataarray['adc_5_probe'].values))
shunt_resistance = shot_dataarray['adc_4_shunt_resistance'].values
print(probe_designations, shunt_resistance)

downsampling_factor = int(shot_dataarray['adc_freqs'].values / DESIRED_DATARATE)
cabling_resistance = (CABLE_RESISTANCES[int(shot_dataarray['adc_4_coax'].values) - 1] + 1.2
                      + PROBE_RESISTANCES[probe_designations[0]])
print(downsampling_factor, cabling_resistance)
acq_time = shot_dataarray['acquisition_length'].values
print('acq_time', acq_time)
print(shot_dataarray['shot_time'].values, shot_dataarray['adc_time'].values, shot_dataarray['shot_end_time'].values, shot_dataarray['adc_end_time'].values)

sweep_range = get_sweep_range(shot_dataarray['shot_end_time'].values, shot_dataarray['adc_end_time'].values, acq_time,
                              shot_dataarray['adc_freqs'].values / downsampling_factor)
print(sweep_range)

print(f'Attempting analysis on shot {shot_number}')

2019-05-28_Leland/
2019-05-28 15h 49m 45s TT_06696074692831398075.adc 6696074692831397888
0.26238938453554556 2.18802022279052e+19
('S', 'L') 10.0
10 3.9
acq_time 5.0
2019-05-28T14:49:35.000000000 2019-05-28T14:49:40.000000000 2019-05-28T14:49:50.000000000 2019-05-28T14:49:45.000000000
(1, 500)
Attempting analysis on shot 0
